In [22]:
import numpy as np
import warnings
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input, Conv2D, MaxPooling2D, Conv3D, MaxPooling3D
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

warnings.filterwarnings('ignore')

In [59]:
ENV_NAME = 'PongNoFrameskip-v4'

# Get the environment and extract the number of actions available in the Cartpole problem
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [60]:
env.observation_space.shape

(210, 160, 3)

In [61]:
model = Sequential()
model.add(Conv3D(1,(1,1,1),input_shape=(1,) + env.observation_space.shape))
model.add(MaxPooling3D((1,4,4)))

model.add(Conv3D(4,(1,3,3)))
model.add(MaxPooling3D((1,4,4)))

model.add(Conv3D(8,(1,3,3)))
model.add(MaxPooling3D((1,2,2)))

# model.add(Conv3D(16,(1,3,3)))
# model.add(MaxPooling3D((1,2,2)))

# model.add(Conv3D(32,(1,3,3)))
# model.add(MaxPooling3D((1,2,2)))

model.add(Flatten())
# model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(nb_actions , activation='linear'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_31 (Conv3D)           (None, 1, 210, 160, 1)    4         
_________________________________________________________________
max_pooling3d_28 (MaxPooling (None, 1, 52, 40, 1)      0         
_________________________________________________________________
conv3d_32 (Conv3D)           (None, 1, 50, 38, 4)      40        
_________________________________________________________________
max_pooling3d_29 (MaxPooling (None, 1, 12, 9, 4)       0         
_________________________________________________________________
conv3d_33 (Conv3D)           (None, 1, 10, 7, 8)       296       
_________________________________________________________________
max_pooling3d_30 (MaxPooling (None, 1, 5, 3, 8)        0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 120)               0         
__________

In [62]:
policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=500000, window_length=1)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this slows down training quite a lot. 
dqn.fit(env, nb_steps=100, visualize=False, verbose=2)

Training for 100 steps ...
done, took 47.776 seconds


In [48]:
dqn.fit(env, nb_steps=100, visualize=False, verbose=2)

Training for 100 steps ...
done, took 11.820 seconds


In [63]:
dqn.test(env, nb_episodes=4, visualize=True)

Testing for 4 episodes ...
Episode 1: reward: -21.000, steps: 3056
Episode 2: reward: -21.000, steps: 3056
Episode 3: reward: -21.000, steps: 3056
Episode 4: reward: -21.000, steps: 3056
